<a href="https://colab.research.google.com/github/delajiqi/awesome/blob/main/%E5%BD%A9%E7%A5%A8%E5%88%86%E6%9E%90%E8%84%9A%E6%9C%AC_(%E5%A4%9A%E7%AD%96%E7%95%A5%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import io
import time
from datetime import datetime
from itertools import product, combinations_with_replacement

# ====================== 配置路径 ======================
# 将输出文件保存到脚本运行目录下的 'output' 文件夹中
output_dir = os.path.join(os.getcwd(), "彩票分析结果")
os.makedirs(output_dir, exist_ok=True) # 确保输出目录存在

# 数据源URL
fc_url = "http://data.17500.cn/3d_desc.txt"
tc_url = "https://data.17500.cn/pl32_desc.txt"

# 生成带时间戳的文件名
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(output_dir, f"多维度分析_{timestamp}.xlsx")

# ====================== 数据加载和预处理 ======================
def load_and_process_from_url(url, lottery_type):
    """
    从指定URL获取数据并进行预处理。
    """
    try:
        headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" }
        print(f"尝试从 {url} 获取数据...")
        # 增加 verify=False 来尝试绕过SSL错误，但这会降低安全性
        response = requests.get(url, headers=headers, timeout=15, verify=False)
        response.raise_for_status()
        content = response.text
        data_io = io.StringIO(content)
        if lottery_type == 'FC':
            col_names = ["开奖期号", "开奖日期", "百位", "十位", "个位", "试机号1", "试机号2", "试机号3", "组选类型", "销量", "直选_中奖注数", "直选_奖金", "组三_中奖注数", "组三_奖金", "组六_中奖注数", "组六_奖金", "其他中奖信息"]
        else:
            col_names = ["开奖期号", "开奖日期", "百位", "十位", "个位", "试机号1", "试机号2", "试机号3", "直选销量", "组选销量", "直选奖金", "组选3奖金", "组选6奖金", "直选_中奖注数", "组三_中奖注数", "组选6_中奖注数", "奖池余额"]

        df = pd.read_csv(data_io, header=None, names=col_names, encoding='utf-8', on_bad_lines='skip', sep='\s+')
        df = df[['开奖期号', '开奖日期', '百位', '十位', '个位']]
        df["开奖日期"] = pd.to_datetime(df["开奖日期"], errors="coerce").dt.strftime("%Y%m%d")
        df = df.sort_values(by=["开奖日期", "开奖期号"], ascending=True).reset_index(drop=True)
        for col in ["百位", "十位", "个位"]:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(-1).astype(int).clip(0, 9)
        valid_mask = (df["百位"] >= 0) & (df["十位"] >= 0) & (df["个位"] >= 0) & (df["开奖日期"].notnull())
        df = df[valid_mask]
        df["开奖号码"] = df.apply(lambda row: f"{row['百位']}-{row['十位']}-{row['个位']}", axis=1)
        return df.drop(columns=["百位", "十位", "个位"])
    except requests.exceptions.RequestException as e:
        print(f"从 URL {url} 获取数据失败: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"处理来自 URL {url} 的数据时发生错误: {e}")
        return pd.DataFrame()

# ====================== 形态分析函数 (大规模扩展) ======================
def analyze_all_shapes(df):
    """
    对开奖号码进行所有维度的形态分析。
    """
    print("正在对所有数据进行全维度形态分析...")
    for prefix in ["福彩", "体彩"]:
        col_name = '3D' if prefix == '福彩' else '排列三'
        # 应用所有形态分析函数
        df[f'{prefix}大中小'] = df[col_name].apply(check_dzx_group)
        df[f'{prefix}012路'] = df[col_name].apply(check_012_road_group)
        df[f'{prefix}大中小细分'] = df[col_name].apply(check_dzx_detailed)
        df[f'{prefix}012路细分'] = df[col_name].apply(check_012_road_detailed)
        df[f'{prefix}大小形态'] = df[col_name].apply(check_size_pattern)
        df[f'{prefix}单双形态'] = df[col_name].apply(check_parity_pattern)
        df[f'{prefix}质合形态'] = df[col_name].apply(check_prime_pattern)
        df[f'{prefix}特殊组合'] = df[col_name].apply(check_special_combo)
        # 新增：带组六过滤的形态分析
        df[f'{prefix}大小形态(组六)'] = df[col_name].apply(check_size_pattern_g6)
        df[f'{prefix}单双形态(组六)'] = df[col_name].apply(check_parity_pattern_g6)
        df[f'{prefix}质合形态(组六)'] = df[col_name].apply(check_prime_pattern_g6)
    return df

# --- 单个号码的形态判断逻辑 ---
def safe_split(s): return list(map(int, str(s).split("-"))) if isinstance(s, str) and '-' in s else []

def check_dzx_group(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    labels = set(('大' if i >= 7 else '中' if i >= 3 else '小') for i in n)
    return "中奖" if len(labels) == 3 else ""

def check_012_road_group(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    return "中奖" if len(set(i % 3 for i in n)) == 3 else ""

def check_dzx_detailed(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    counts = {'大': 0, '中': 0, '小': 0}
    for i in n: counts['大' if i >= 7 else '中' if i >= 3 else '小'] += 1
    if counts['大']==2 and counts['中']==1: return '大大中'
    if counts['大']==2 and counts['小']==1: return '大大小'
    if counts['中']==2 and counts['小']==1: return '中中小'
    if counts['中']==2 and counts['大']==1: return '中中大'
    if counts['小']==2 and counts['中']==1: return '小小中'
    if counts['小']==2 and counts['大']==1: return '小小大'
    return ""

def check_012_road_detailed(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    counts = {0: 0, 1: 0, 2: 0}
    for i in n: counts[i % 3] += 1
    if counts[0]==2 and counts[1]==1: return '001路'
    if counts[0]==2 and counts[2]==1: return '002路'
    if counts[1]==2 and counts[0]==1: return '110路'
    if counts[1]==2 and counts[2]==1: return '112路'
    if counts[2]==2 and counts[0]==1: return '220路'
    if counts[2]==2 and counts[1]==1: return '221路'
    return ""

def check_size_pattern(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    large_count = sum(1 for i in n if i >= 5)
    if large_count == 3: return '全大'
    if large_count == 2: return '两大一小'
    if large_count == 1: return '两小一大'
    if large_count == 0: return '全小'
    return ""

def check_parity_pattern(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    odd_count = sum(1 for i in n if i % 2 != 0)
    if odd_count == 3: return '全单'
    if odd_count == 2: return '两单一双'
    if odd_count == 1: return '两双一单'
    if odd_count == 0: return '全双'
    return ""

def check_prime_pattern(s):
    n = safe_split(s)
    if len(n) != 3: return ""
    primes = {1, 2, 3, 5, 7}
    prime_count = sum(1 for i in n if i in primes)
    if prime_count == 3: return '全质'
    if prime_count == 2: return '两质一合'
    if prime_count == 1: return '两合一质'
    if prime_count == 0: return '全合'
    return ""

def check_special_combo(s):
    n = safe_split(s)
    if len(n) != 3 or len(set(n)) != 3: return ""
    sum_val = sum(n)
    span_val = max(n) - min(n)
    if 9 <= sum_val <= 17 and 3 <= span_val <= 7: return "中奖"
    return ""

# --- 新增：带组六过滤的判断逻辑 ---
def check_size_pattern_g6(s):
    n = safe_split(s)
    if len(set(n)) != 3: return "" # 必须是组六
    return check_size_pattern(s) + '(G6)' if check_size_pattern(s) else ""

def check_parity_pattern_g6(s):
    n = safe_split(s)
    if len(set(n)) != 3: return ""
    return check_parity_pattern(s) + '(G6)' if check_parity_pattern(s) else ""

def check_prime_pattern_g6(s):
    n = safe_split(s)
    if len(set(n)) != 3: return ""
    return check_prime_pattern(s) + '(G6)' if check_prime_pattern(s) else ""

# ====================== 核心计算引擎 (可重用) ======================
def compute_sequential_omission(df, fc_res_col, tc_res_col, fc_om_col, tc_om_col):
    counter, fc_omissions, tc_omissions = 0, [], []
    for _, row in df.iterrows():
        if row.get(fc_res_col): fc_omissions.append("中奖"); counter = 0
        else: counter += 1; fc_omissions.append(counter)
        if row.get(tc_res_col): tc_omissions.append("中奖"); counter = 0
        else: counter += 1; tc_omissions.append(counter)
    df[fc_om_col], df[tc_om_col] = fc_omissions, tc_omissions
    return df

def analyze_kill_strategy(df, history_count, strat_name, fc_base_col, tc_base_col, fc_num_col, tc_num_col):
    print(f"正在执行 {strat_name} 分析 (基于 {fc_base_col.replace('福彩','')} - 杀 {history_count*2} 期)...")
    fc_hist, tc_hist, fc_res, tc_res, counter = [], [], [], [], 0
    for _, row in df.iterrows():
        killed = set(fc_hist[-history_count:] + tc_hist[-history_count:])
        is_fc_win, fc_num = row.get(fc_base_col), row.get(fc_num_col)
        if fc_num and is_fc_win:
            norm_num = "".join(sorted(fc_num.split('-')))
            if norm_num not in killed: fc_res.append('中奖'); counter = 0
            else: counter += 1; fc_res.append(counter)
            fc_hist.append(norm_num)
        else: counter += 1; fc_res.append(counter)

        killed = set(fc_hist[-history_count:] + tc_hist[-history_count:])
        is_tc_win, tc_num = row.get(tc_base_col), row.get(tc_num_col)
        if tc_num and is_tc_win:
            norm_num = "".join(sorted(tc_num.split('-')))
            if norm_num not in killed: tc_res.append('中奖'); counter = 0
            else: counter += 1; tc_res.append(counter)
            tc_hist.append(norm_num)
        else: counter += 1; tc_res.append(counter)

    df[f'{strat_name}福彩'], df[f'{strat_name}体彩'] = fc_res, tc_res
    return df, set(fc_hist[-history_count:] + tc_hist[-history_count:])

# ====================== 新增: 历史遗漏排名函数 ======================
def find_top_omission_periods(df, fc_om_col, tc_om_col):
    """
    找出指定遗漏列中，历史最长的前10个遗漏周期。
    """
    omission_stream = []
    for index, row in df.iterrows():
        omission_stream.append({'date': row['开奖日期'], 'value': row[fc_om_col]})
        omission_stream.append({'date': row['开奖日期'], 'value': row[tc_om_col]})

    periods = []
    current_period = None

    for i, item in enumerate(omission_stream):
        value = item['value']
        if isinstance(value, (int, float)) and np.isreal(value):
            if current_period is None:
                current_period = {'start_date': item['date'], 'end_date': item['date'], 'length': value}
            else:
                current_period['end_date'] = item['date']
                current_period['length'] = value
        else:
            if current_period is not None:
                periods.append(current_period)
            current_period = None

    if current_period is not None:
        periods.append(current_period)

    return sorted(periods, key=lambda x: x['length'], reverse=True)[:10]

# ====================== 组合生成函数 ======================
def generate_all_combinations():
    """生成所有220种组选组合(豹子、组三、组六)。"""
    return {"".join(map(str, sorted(c))) for c in combinations_with_replacement(range(10), 3)}

def get_combinations_for_pattern(all_combos, pattern_func, win_conditions):
    """根据形态函数和中奖条件筛选组合。"""
    return {c for c in all_combos if pattern_func("-".join(c)) in win_conditions}

# ====================== Excel格式设置 (已重构) ======================
def add_excel_formatting(writer, sheet_name, df, summary_data, top_omissions_report, omission_cols):
    """为指定工作表添加格式和策略小结。"""
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # 默认显示最后一行功能已暂时移除，以确保代码稳定运行
    worksheet.freeze_panes(1, 4)
    # 定义格式
    header_fmt = workbook.add_format({'bold': True, 'bg_color': '#D9D9D9', 'border': 1, 'align': 'center', 'valign': 'vcenter'})
    yellow_fmt = workbook.add_format({'bg_color': '#FFFF00'})
    green_fmt  = workbook.add_format({'bg_color': '#92D050'})
    red_fmt    = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
    purple_fmt = workbook.add_format({'bg_color': '#7030A0', 'font_color': '#FFFFFF'})
    summary_header_fmt = workbook.add_format({'bold': True, 'bg_color': '#4F81BD', 'font_color': 'white', 'border': 1, 'align': 'center', 'text_wrap': True})
    summary_text_fmt = workbook.add_format({'text_wrap': True, 'valign': 'top', 'border': 1})
    leopard_fmt = workbook.add_format({'bg_color': '#ADD8E6'})
    group3_fmt = workbook.add_format({'bg_color': '#FFFF00'})
    report_header_fmt = workbook.add_format({'bold': True, 'bg_color': '#F79646', 'font_color': 'white', 'border': 1, 'align': 'center'})
    report_subheader_fmt = workbook.add_format({'bold': True, 'bg_color': '#C0504D', 'font_color': 'white', 'border': 1, 'align': 'center'})
    report_cell_fmt = workbook.add_format({'border': 1, 'align': 'center'})

    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_fmt)

    for idx, col_name in enumerate(df.columns):
        series = df[col_name]
        if col_name == '开奖日期':
            worksheet.set_column(idx, idx, 12)
        else:
            max_len = max((series.astype(str).map(len).max()), len(str(col_name))) + 2
            worksheet.set_column(idx, idx, max_len)

        if col_name in ['3D', '排列三']:
            col_letter = chr(ord('A') + idx)
            cell_range = f"{col_letter}2:{col_letter}{len(df) + 1}"
            worksheet.conditional_format(cell_range, {'type': 'formula', 'criteria': f'=AND(LEN({col_letter}2)=5, MID({col_letter}2,1,1)=MID({col_letter}2,3,1), MID({col_letter}2,3,1)=MID({col_letter}2,5,1))', 'format': leopard_fmt})
            worksheet.conditional_format(cell_range, {'type': 'formula', 'criteria': f'=AND(LEN({col_letter}2)=5, OR(MID({col_letter}2,1,1)=MID({col_letter}2,3,1), MID({col_letter}2,1,1)=MID({col_letter}2,5,1), MID({col_letter}2,3,1)=MID({col_letter}2,5,1)), NOT(AND(MID({col_letter}2,1,1)=MID({col_letter}2,3,1), MID({col_letter}2,3,1)=MID({col_letter}2,5,1))))', 'format': group3_fmt})

    for col_name in omission_cols:
        if col_name not in df.columns: continue
        col_idx = df.columns.get_loc(col_name)
        col_letter = chr(ord('A') + col_idx)
        cell_range = f"{col_letter}2:{col_letter}{len(df) + 1}"
        worksheet.conditional_format(cell_range, {'type': 'formula', 'criteria': f'=AND(ISNUMBER({col_letter}2), {col_letter}2>30)', 'format': purple_fmt})
        worksheet.conditional_format(cell_range, {'type': 'formula', 'criteria': f'=AND(ISNUMBER({col_letter}2), {col_letter}2>=21, {col_letter}2<=30)', 'format': red_fmt})
        worksheet.conditional_format(cell_range, {'type': 'formula', 'criteria': f'=AND(ISNUMBER({col_letter}2), {col_letter}2>=11, {col_letter}2<=20)', 'format': green_fmt})
        worksheet.conditional_format(cell_range, {'type': 'formula', 'criteria': f'=AND(ISNUMBER({col_letter}2), {col_letter}2>0, {col_letter}2<10)', 'format': yellow_fmt})

    current_write_row = len(df) + 3
    worksheet.merge_range(current_write_row, 0, current_write_row, len(df.columns)-1, "策略小结 (下次投注参考)", header_fmt)
    current_write_row += 1
    for i, (strategy_name, data) in enumerate(summary_data.items()):
        worksheet.write(current_write_row, 0, f"{strategy_name}杀去组合 ({len(data['killed'])}注):", summary_header_fmt)
        worksheet.merge_range(current_write_row, 1, current_write_row, len(df.columns)-1, ", ".join(sorted(list(data['killed']))), summary_text_fmt)
        current_write_row += 1
        worksheet.write(current_write_row, 0, f"{strategy_name}剩余组合 ({len(data['remaining'])}注):", summary_header_fmt)
        worksheet.merge_range(current_write_row, 1, current_write_row, len(df.columns)-1, ", ".join(sorted(list(data['remaining']))), summary_text_fmt)
        current_write_row += 1

    current_write_row += 1
    worksheet.merge_range(current_write_row, 0, current_write_row, len(df.columns)-1, "历史最长遗漏Top 10", report_header_fmt)
    current_write_row += 1

    report_start_col = 0
    for report_name, report_data in top_omissions_report.items():
        if report_start_col + 4 > 16:
             current_write_row += 13
             report_start_col = 0

        worksheet.merge_range(current_write_row, report_start_col, current_write_row, report_start_col + 3, report_name, report_subheader_fmt)
        worksheet.write(current_write_row + 1, report_start_col, "排名", header_fmt)
        worksheet.write(current_write_row + 1, report_start_col + 1, "遗漏长度", header_fmt)
        worksheet.write(current_write_row + 1, report_start_col + 2, "开始日期", header_fmt)
        worksheet.write(current_write_row + 1, report_start_col + 3, "结束日期", header_fmt)

        worksheet.set_column(report_start_col, report_start_col, 6)
        worksheet.set_column(report_start_col+1, report_start_col+1, 10)
        worksheet.set_column(report_start_col+2, report_start_col+3, 12)

        for rank, period in enumerate(report_data, 1):
            worksheet.write(current_write_row + 1 + rank, report_start_col, rank, report_cell_fmt)
            worksheet.write(current_write_row + 1 + rank, report_start_col + 1, period['length'], report_cell_fmt)
            worksheet.write(current_write_row + 1 + rank, report_start_col + 2, period['start_date'], report_cell_fmt)
            worksheet.write(current_write_row + 1 + rank, report_start_col + 3, period['end_date'], report_cell_fmt)
        report_start_col += 5


# ====================== 主程序 ======================
if __name__ == "__main__":
    df_fc = load_and_process_from_url(fc_url, 'FC')
    time.sleep(np.random.randint(2, 6))
    df_tc = load_and_process_from_url(tc_url, 'TC')

    # FIX: Robustly handle cases where one DataFrame might be empty
    if df_fc.empty and df_tc.empty:
        print("无法获取任何数据，程序终止。"); exit()
    elif not df_fc.empty and not df_tc.empty:
        merged_df = pd.merge(df_fc, df_tc, on="开奖日期", suffixes=('_福彩', '_体彩'), how="outer").sort_values(by="开奖日期").reset_index(drop=True)
    elif not df_fc.empty:
        merged_df = df_fc.copy()
    else: # Only df_tc is not empty
        merged_df = df_tc.copy()

    merged_df['开奖期号'] = merged_df.get('开奖期号_福彩', pd.Series(dtype=str)).fillna(merged_df.get('开奖期号_体彩', pd.Series(dtype=str)))
    merged_df = merged_df.rename(columns={"开奖号码_福彩": "3D", "开奖号码_体彩": "排列三"})
    merged_df = merged_df.fillna("").drop(columns=['开奖期号_福彩', '开奖期号_体彩'], errors='ignore')

    # Ensure essential columns exist, even if one source failed
    for col in ['3D', '排列三']:
        if col not in merged_df:
            merged_df[col] = ""

    merged_df = analyze_all_shapes(merged_df)

    # --- 定义所有分析任务 ---
    all_combos = generate_all_combinations()
    analysis_tasks = [
        {'sheet_name': '大中小', 'base_col': '大中小', 'win_cond': ['中奖'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_group, ['中奖'])},
        {'sheet_name': '大大中', 'base_col': '大中小细分', 'win_cond': ['大大中'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_detailed, ['大大中'])},
        {'sheet_name': '大大小', 'base_col': '大中小细分', 'win_cond': ['大大小'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_detailed, ['大大小'])},
        {'sheet_name': '中中小', 'base_col': '大中小细分', 'win_cond': ['中中小'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_detailed, ['中中小'])},
        {'sheet_name': '中中大', 'base_col': '大中小细分', 'win_cond': ['中中大'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_detailed, ['中中大'])},
        {'sheet_name': '小小中', 'base_col': '大中小细分', 'win_cond': ['小小中'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_detailed, ['小小中'])},
        {'sheet_name': '小小大', 'base_col': '大中小细分', 'win_cond': ['小小大'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_dzx_detailed, ['小小大'])},
        {'sheet_name': '012路', 'base_col': '012路', 'win_cond': ['中奖'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_group, ['中奖'])},
        {'sheet_name': '001路', 'base_col': '012路细分', 'win_cond': ['001路'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_detailed, ['001路'])},
        {'sheet_name': '002路', 'base_col': '012路细分', 'win_cond': ['002路'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_detailed, ['002路'])},
        {'sheet_name': '110路', 'base_col': '012路细分', 'win_cond': ['110路'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_detailed, ['110路'])},
        {'sheet_name': '112路', 'base_col': '012路细分', 'win_cond': ['112路'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_detailed, ['112路'])},
        {'sheet_name': '220路', 'base_col': '012路细分', 'win_cond': ['220路'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_detailed, ['220路'])},
        {'sheet_name': '221路', 'base_col': '012路细分', 'win_cond': ['221路'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_012_road_detailed, ['221路'])},
        {'sheet_name': '全大', 'base_col': '大小形态', 'win_cond': ['全大'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_size_pattern, ['全大'])},
        {'sheet_name': '两大一小', 'base_col': '大小形态', 'win_cond': ['两大一小'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_size_pattern, ['两大一小'])},
        {'sheet_name': '两小一大', 'base_col': '大小形态', 'win_cond': ['两小一大'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_size_pattern, ['两小一大'])},
        {'sheet_name': '全小', 'base_col': '大小形态', 'win_cond': ['全小'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_size_pattern, ['全小'])},
        {'sheet_name': '全单', 'base_col': '单双形态', 'win_cond': ['全单'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_parity_pattern, ['全单'])},
        {'sheet_name': '两单一双', 'base_col': '单双形态', 'win_cond': ['两单一双'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_parity_pattern, ['两单一双'])},
        {'sheet_name': '两双一单', 'base_col': '单双形态', 'win_cond': ['两双一单'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_parity_pattern, ['两双一单'])},
        {'sheet_name': '全双', 'base_col': '单双形态', 'win_cond': ['全双'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_parity_pattern, ['全双'])},
        {'sheet_name': '全质', 'base_col': '质合形态', 'win_cond': ['全质'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_prime_pattern, ['全质'])},
        {'sheet_name': '两质一合', 'base_col': '质合形态', 'win_cond': ['两质一合'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_prime_pattern, ['两质一合'])},
        {'sheet_name': '两合一质', 'base_col': '质合形态', 'win_cond': ['两合一质'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_prime_pattern, ['两合一质'])},
        {'sheet_name': '全合', 'base_col': '质合形态', 'win_cond': ['全合'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_prime_pattern, ['全合'])},
        {'sheet_name': '特殊组合', 'base_col': '特殊组合', 'win_cond': ['中奖'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_special_combo, ['中奖'])},
        # 新增: 带组六过滤的分析任务
        {'sheet_name': '两大一小(G6)', 'base_col': '大小形态(组六)', 'win_cond': ['两大一小(G6)'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_size_pattern_g6, ['两大一小(G6)'])},
        {'sheet_name': '两小一大(G6)', 'base_col': '大小形态(组六)', 'win_cond': ['两小一大(G6)'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_size_pattern_g6, ['两小一大(G6)'])},
        {'sheet_name': '两单一双(G6)', 'base_col': '单双形态(组六)', 'win_cond': ['两单一双(G6)'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_parity_pattern_g6, ['两单一双(G6)'])},
        {'sheet_name': '两双一单(G6)', 'base_col': '单双形态(组六)', 'win_cond': ['两双一单(G6)'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_parity_pattern_g6, ['两双一单(G6)'])},
        {'sheet_name': '两质一合(G6)', 'base_col': '质合形态(组六)', 'win_cond': ['两质一合(G6)'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_prime_pattern_g6, ['两质一合(G6)'])},
        {'sheet_name': '两合一质(G6)', 'base_col': '质合形态(组六)', 'win_cond': ['两合一质(G6)'], 'combo_func': lambda: get_combinations_for_pattern(all_combos, check_prime_pattern_g6, ['两合一质(G6)'])},
    ]

    # 筛选2005年及以后的数据
    merged_df['开奖日期'] = pd.to_datetime(merged_df['开奖日期'], errors='coerce')
    merged_df = merged_df[merged_df['开奖日期'] >= '2005-01-01'].copy()
    merged_df['开奖日期'] = merged_df['开奖日期'].dt.strftime('%Y-%m-%d')

    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        for task in analysis_tasks:
            print(f"\n--- 开始 {task['sheet_name']} 分析 ---")
            temp_df = merged_df.copy()

            # --- 基础遗漏 ---
            fc_res_col, tc_res_col = f"福彩{task['base_col']}", f"体彩{task['base_col']}"
            fc_om_col, tc_om_col = f"福彩{task['base_col']}遗漏", f"体彩{task['base_col']}遗漏"
            temp_df['CURRENT_FC_WIN'] = temp_df[fc_res_col].apply(lambda x: "中奖" if x in task['win_cond'] else "")
            temp_df['CURRENT_TC_WIN'] = temp_df[tc_res_col].apply(lambda x: "中奖" if x in task['win_cond'] else "")
            temp_df = compute_sequential_omission(temp_df, 'CURRENT_FC_WIN', 'CURRENT_TC_WIN', fc_om_col, tc_om_col)

            # --- 杀号策略 ---
            summary_data, top_omissions_report = {}, {}
            top_omissions_report['不杀号'] = find_top_omission_periods(temp_df, fc_om_col, tc_om_col)

            task_combos = task['combo_func']()
            for h_count, h_name in [(4, "杀8期"), (5, "杀10期"), (6, "杀12期")]:
                strat_name = f"{h_name}策略"
                temp_df, killed_set = analyze_kill_strategy(temp_df, h_count, strat_name, 'CURRENT_FC_WIN', 'CURRENT_TC_WIN', '3D', '排列三')
                summary_data[h_name] = {'killed': killed_set, 'remaining': task_combos - killed_set}
                top_omissions_report[h_name] = find_top_omission_periods(temp_df, f"{strat_name}福彩", f"{strat_name}体彩")

            # --- 准备输出 ---
            base_cols = ["开奖日期", "开奖期号", "3D", "排列三"]
            omission_cols = [fc_om_col, tc_om_col]
            for h_name in ["杀8期", "杀10期", "杀12期"]:
                omission_cols.append(f"{h_name}策略福彩")
                omission_cols.append(f"{h_name}策略体彩")

            sheet_df = temp_df[base_cols + omission_cols]

            # --- 写入Sheet ---
            sheet_df.to_excel(writer, index=False, sheet_name=task['sheet_name'])
            add_excel_formatting(writer, task['sheet_name'], sheet_df, summary_data, top_omissions_report, omission_cols)

    print(f"\n分析完成！已生成包含多个分析维度的Excel文件：{output_file}")